<a href="https://colab.research.google.com/github/KayalvizhiT513/TNEA-seat-matrix-analysis/blob/main/TNEA_seat_matrix_running_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pdfplumber pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 75.2 MB/s eta 0:00:00


In [2]:
import os
import pdfplumber
import pandas as pd

# Folder containing the PDFs
pdf_folder = '/content/drive/MyDrive/TNEA Seat Matrix 2025'

# Output folder for Excel files
output_folder = pdf_folder

In [3]:
# List to hold one DataFrame per PDF
pdf_dataframes = []

# Process each PDF
for filename in os.listdir(pdf_folder):
    if filename.endswith('general_academic_seat_matrix_2024-no-1.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)

        combined_df = pd.DataFrame()

        # Extract tables and merge into one DataFrame
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                tables = page.extract_tables()
                for table in tables:
                    df = pd.DataFrame(table[1:], columns=table[0])  # Skip header row
                    combined_df = pd.concat([combined_df, df], ignore_index=True)

        # Append the combined DataFrame to the list
        pdf_dataframes.append(combined_df)
        print(f'Loaded: {filename} → {combined_df.shape[0]} rows')

Loaded: general_academic_seat_matrix_2024-no-1.pdf → 3469 rows


In [4]:
# Replace \n in column names with a space
pdf_dataframes[0].columns = pdf_dataframes[0].columns.str.replace('\n', ' ')
# pdf_dataframes[1].columns = pdf_dataframes[1].columns.str.replace('\n', ' ')
# pdf_dataframes[2].columns = pdf_dataframes[1].columns.str.replace('\n', ' ')

In [5]:
pdf_dataframes[0]

,COLLEGE CODE,COLLEGE NAME,BRANCH CODE,BRANCH NAME,OC,BC,BCM,MBC,SC,SCA,ST,TOTAL
0,1,University Departments of Anna University Chen...,BY,BIO MEDICAL ENGINEERING\n(SS),17,14,2,10,8,2,0,53
1,1,University Departments of Anna University Chen...,CE,CIVIL ENGINEERING,16,14,2,11,8,2,0,53
2,1,University Departments of Anna University Chen...,CM,COMPUTER SCIENCE AND\nENGINEERING (SS),33,29,4,21,16,4,0,107
3,1,University Departments of Anna University Chen...,CS,COMPUTER SCIENCE AND\nENGINEERING,17,14,2,11,7,2,1,54
4,1,University Departments of Anna University Chen...,EC,ELECTRONICS AND\nCOMMUNICATION\nENGINEERING,17,14,2,11,8,2,0,54
...,...,...,...,...,...,...,...,...,...,...,...,...
3464,5990,"Latha Mathavan Engineering College, Latha Math...",CE,CIVIL ENGINEERING,14,11,2,8,7,2,0,44
3465,5990,"Latha Mathavan Engineering College, Latha Math...",CS,COMPUTER SCIENCE AND\nENGINEERING,22,19,2,15,10,3,0,71
3466,5990,"Latha Mathavan Engineering College, Latha Math...",EC,ELECTRONICS AND\nCOMMUNICATION\nENGINEERING,12,9,1,7,5,1,1,36
3467,5990,"Latha Mathavan Engineering College, Latha Math...",EE,ELECTRICAL AND ELECTRONICS\nENGINEERING,8,7,1,6,3,0,1,26


In [9]:
pdf_dataframes[0]['COLLEGE CODE'] = pdf_dataframes[0]['COLLEGE CODE'].astype(int)

In [10]:
pdf_dataframes[0].to_excel("/content/drive/MyDrive/TNEA Seat Matrix 2025/general_academic_seat_matrix_2024-no-1.xlsx", index=False)

In [ ]:
# compare pdf_dataframes[1] and pdf_dataframes[2] with columns 'COLLEGE CODE' + 'BRANCH CODE' and find the mismatched rows
import pandas as pd
from IPython.display import display
import re

# Create unique keys for both DataFrames
df1 = pdf_dataframes[1].copy() # before spl reservation counceling
df2 = pdf_dataframes[2].copy() # after spl reservation counceling

# Branch code is 2 length code in caps anything trailing or preceeding need to be removed
df1['BRANCH CODE'] = df1['BRANCH CODE'].astype(str).str.findall(r'[A-Z]{2}').str[0].fillna('')
df2['BRANCH CODE'] = df2['BRANCH CODE'].astype(str).str.findall(r'[A-Z]{2}').str[0].fillna('')

df1['KEY'] = df1['COLLEGE CODE'].astype(str) + '_' + df1['BRANCH CODE'].astype(str)
df2['KEY'] = df2['COLLEGE CODE'].astype(str) + '_' + df2['BRANCH CODE'].astype(str)

# Find mismatched keys
keys1 = set(df1['KEY'])
keys2 = set(df2['KEY'])

only_in_df1_keys = keys1 - keys2
only_in_df2_keys = keys2 - keys1

# Extract mismatched rows
mismatched_df1 = df1[df1['KEY'].isin(only_in_df1_keys)]
mismatched_df2 = df2[df2['KEY'].isin(only_in_df2_keys)]

# Display mismatches
mismatched_df1.reset_index(drop=True, inplace=True)
mismatched_df2.reset_index(drop=True, inplace=True)

# Optionally, drop the KEY column after comparison
mismatched_df1.drop(columns='KEY', inplace=True)
mismatched_df2.drop(columns='KEY', inplace=True)

# Output mismatched rows

/tmp/ipython-input-14-2065332089.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatched_df1.drop(columns='KEY', inplace=True)
/tmp/ipython-input-14-2065332089.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatched_df2.drop(columns='KEY', inplace=True)


In [ ]:
df1 = pdf_dataframes[1].copy()

In [ ]:
display(mismatched_df1)

,COLLEGE CODE,COLLEGE NAME,BRANCH CODE,BRANCH NAME,OC,BC,BCM,MBC,SC,SCA,ST,TOTAL


In [ ]:
display(mismatched_df2)

,COLLEGE CODE,COLLEGE NAME,BRANCH CODE,BRANCH NAME,OC,BC,BCM,MBC,SC,SCA,ST,TOTAL
0,1203,"MADRAS ENGINEERING COLLEGE, TAMBARAM ROAD,\nVE...",AD,Artificial Intelligence and Data\nScience,18,13,2,11,9,0,1,54
1,1203,"MADRAS ENGINEERING COLLEGE, TAMBARAM ROAD,\nVE...",AM,COMPUTER SCIENCE AND\nENGINEERING (AI AND MACH...,17,15,2,10,8,2,1,55
2,1203,"MADRAS ENGINEERING COLLEGE, TAMBARAM ROAD,\nVE...",CB,COMPUTER SCIENCE AND\nBUSSINESS SYSTEM,17,15,1,11,9,2,0,55
3,1203,"MADRAS ENGINEERING COLLEGE, TAMBARAM ROAD,\nVE...",CS,COMPUTER SCIENCE AND\nENGINEERING,16,15,2,11,7,2,1,54
4,1203,"MADRAS ENGINEERING COLLEGE, TAMBARAM ROAD,\nVE...",IT,INFORMATION TECHNOLOGY,16,15,2,11,8,1,1,54
5,1203,"MADRAS ENGINEERING COLLEGE, TAMBARAM ROAD,\nVE...",SC,Computer Science and\nEngineering (Cyber Secur...,17,14,2,11,8,2,0,54
6,1339,MAHALAKSHMI TECH CAMPUS (FORMERLY\nMAHALAKSHMI...,AD,Artificial Intelligence and Data\nScience,11,9,1,7,6,1,0,35
7,1339,MAHALAKSHMI TECH CAMPUS (FORMERLY\nMAHALAKSHMI...,AM,COMPUTER SCIENCE AND\nENGINEERING (AI AND MACH...,10,9,2,7,5,1,1,35
8,1339,MAHALAKSHMI TECH CAMPUS (FORMERLY\nMAHALAKSHMI...,CS,COMPUTER SCIENCE AND\nENGINEERING,22,20,2,14,12,2,0,72
9,1339,MAHALAKSHMI TECH CAMPUS (FORMERLY\nMAHALAKSHMI...,EC,ELECTRONICS AND\nCOMMUNICATION ENGINEERING,11,9,1,7,6,1,0,35


In [ ]:
clg_added_after_spl = set(mismatched_df2['COLLEGE CODE'])
len(clg_added_after_spl)

10

In [ ]:
# Find the intersection of the sets
common_colleges = set(df1['COLLEGE CODE']).intersection(set(mismatched_df2['COLLEGE CODE']))

# Display the result
print(f"Number of common colleges: {len(common_colleges)}")
print(f"Common colleges: {common_colleges}")

Number of common colleges: 2
Common colleges: {'3855', '1407'}


In [ ]:
print(f"Number of colleges Before Special Reservation Counselling: {len(df1['COLLEGE CODE'].unique())}")
print(f"Number of colleges After Special Reservation Counselling: {len(df2['COLLEGE CODE'].unique())}")

Number of colleges Before Special Reservation Counselling: 417
Number of colleges After Special Reservation Counselling: 425


In [11]:
cs_branch_codes = [
    "CS",  # Computer Science and Engineering
    "CM",  # CSE (Self-Supporting)
    "XS",  # CSE (Tamil Medium)
    "AL",  # AI & ML
    "AM",  # CSE (AI & ML)
    "CG",  # CSE (AI & ML) (SS)
    "CF",  # CSE (Data Science)
    "CI",  # CSE (IoT)
    "IC",  # CSE (IoT + Cybersecurity + Blockchain)
    "SC",  # Computer Science and Engineering (Cyber Security)
    "CD",  # Computer Science and Design
    "CB",  # CS & Business Systems
    "CW",  # CS & Business Systems (SS)
    "CO",  # Computer and Communication Engineering
    "TS",  # Computer Science and Technology
    "CT",  # Computer Technology
    "CY",  # Cyber Security
    "IT",  # Information Technology
    "IM",  # Information Technology (SS)
    "IS",  # Information Science and Engineering
    "CJ",  # M.Tech CSE (5 Years)
    "AT",  # Artificial Intelligence and Data Science (Self-Supporting)
    "AD",  # Artificial Intelligence and Data Science
]

print(sorted(cs_branch_codes))
len(cs_branch_codes)

['AD', 'AL', 'AM', 'AT', 'CB', 'CD', 'CF', 'CG', 'CI', 'CJ', 'CM', 'CO', 'CS', 'CT', 'CW', 'CY', 'IC', 'IM', 'IS', 'IT', 'SC', 'TS', 'XS']


23

In [12]:
cutoff_OC_CS_df_org = pd.read_excel('/content/drive/MyDrive/TNEA Seat Matrix 2025/CSE(OC).xlsx')
cutoff_OC_CS_df_org

,College Code,College Name,Course Code,OC 2024,OC 2023,OC 2022,OC 2021,OC AVG,PUBLIC PERCEPTION RANK
0,1,"University Departments of Anna University, Che...",CS,200.0,200.0,200.0,199.000,199.75,1
1,1,"University Departments of Anna University, Che...",CM,199.5,199.5,199.5,198.000,199.13,2
2,2006,"PSG College of Technology (Autonomous), Peelam...",CM,198.5,199.0,199.5,197.500,198.63,3
3,4,"University Departments of Anna University, Che...",CS,199.0,199.0,199.0,197.175,198.54,4
4,1315,Sri Sivasubramaniya Nadar College of Engineeri...,CS,198.5,199.0,198.5,197.000,198.25,5
...,...,...,...,...,...,...,...,...,...
404,1445,"Sri Rangapoopathi College of Engineering, Alam...",CS,NaN,NaN,NaN,87.500,87.50,405
405,3854,Mahath Amma Institute of Engineering and Techn...,CS,NaN,NaN,86.5,NaN,86.50,406
406,1524,"Annamalaiar College of Engineering, Modaiyur V...",CS,86.0,NaN,NaN,NaN,86.00,407
407,2341,"Ganesh College of Engineering, Attur Main Road...",CS,85.0,NaN,87.0,NaN,86.00,408


In [13]:
cutoff_OC_CS_df_org.columns

Index(['College Code', 'College Name', 'Course Code', 'OC 2024', 'OC 2023',
       'OC 2022', 'OC 2021', 'OC AVG', 'PUBLIC PERCEPTION RANK'],
      dtype='object')

In [14]:
# drop all columns in avg_cutoff_df except ['College Code', 'College Name', 'Course Code', 'Course Name', 'OC Average']
cutoff_OC_CS_df = cutoff_OC_CS_df_org[['College Code', 'College Name', 'Course Code', 'OC AVG']]
cutoff_OC_CS_df

,College Code,College Name,Course Code,OC AVG
0,1,"University Departments of Anna University, Che...",CS,199.75
1,1,"University Departments of Anna University, Che...",CM,199.13
2,2006,"PSG College of Technology (Autonomous), Peelam...",CM,198.63
3,4,"University Departments of Anna University, Che...",CS,198.54
4,1315,Sri Sivasubramaniya Nadar College of Engineeri...,CS,198.25
...,...,...,...,...
404,1445,"Sri Rangapoopathi College of Engineering, Alam...",CS,87.50
405,3854,Mahath Amma Institute of Engineering and Techn...,CS,86.50
406,1524,"Annamalaiar College of Engineering, Modaiyur V...",CS,86.00
407,2341,"Ganesh College of Engineering, Attur Main Road...",CS,86.00


In [ ]:
print(f"Number of repeated collegs: {len(cutoff_OC_CS_df['College Code']) - len(cutoff_OC_CS_df['College Code'].unique())}")

Number of repeated collegs: 4


In [ ]:
duplicates = cutoff_OC_CS_df['College Code'][cutoff_OC_CS_df['College Code'].duplicated()].unique()
duplicates

array([   1,    4, 2007, 5008])

In [ ]:
cutoff_OC_CS_df_org[cutoff_OC_CS_df_org['College Code'].isin(duplicates)]

,College Code,College Name,Course Code,OC 2024,OC 2023,OC 2022,OC 2021,OC AVG,PUBLIC PERCEPTION RANK
0,1,"University Departments of Anna University, Che...",CS,200.0,200.0,200.0,199.000,199.75,1
1,1,"University Departments of Anna University, Che...",CM,199.5,199.5,199.5,198.000,199.13,2
3,4,"University Departments of Anna University, Che...",CS,199.0,199.0,199.0,197.175,198.54,4
5,4,"University Departments of Anna University, Che...",CM,198.0,198.5,198.5,196.500,197.88,6
6,5008,Thiagarajar College of Engineering (Autonomous...,CS,196.5,198.0,198.0,195.020,196.88,7
7,2007,Coimbatore Institute of Technology (Autonomous...,CS,197.0,197.0,197.0,194.900,196.48,8
10,2007,Coimbatore Institute of Technology (Autonomous...,CM,196.0,196.5,196.5,194.365,195.84,11
11,5008,Thiagarajar College of Engineering (Autonomous...,CM,194.5,196.0,196.5,194.155,195.29,12


In [15]:
unique_clg_ppr_order = cutoff_OC_CS_df['College Code'].unique()

In [ ]:
df1['COLLEGE CODE'] = df1['COLLEGE CODE'].astype(str).str.strip()
cutoff_OC_CS_df['College Code'] = cutoff_OC_CS_df['College Code'].astype(str).str.strip()

In [16]:
# OC	BC	BCM	MBC	SC	SCA	ST
categories = ['OC', 'BC', 'BCM', 'MBC', 'SC', 'SCA', 'ST']

In [ ]:
df1[df1['COLLEGE CODE'] == '1']

,COLLEGE CODE,COLLEGE NAME,BRANCH CODE,BRANCH NAME,OC,BC,BCM,MBC,SC,SCA,ST,TOTAL
0,1,University Departments of Anna University Chen...,BY,BIO MEDICAL ENGINEERING\n(SS),19,17,3,13,9,2,1,64
1,1,University Departments of Anna University Chen...,CE,CIVIL ENGINEERING,19,18,2,13,9,2,1,64
2,1,University Departments of Anna University Chen...,CM,COMPUTER SCIENCE AND\nENGINEERING (SS),40,33,4,25,19,4,1,126
3,1,University Departments of Anna University Chen...,CS,COMPUTER SCIENCE AND\nENGINEERING,19,17,2,13,9,2,1,63
4,1,University Departments of Anna University Chen...,EC,ELECTRONICS AND\nCOMMUNICATION\nENGINEERING,19,17,2,13,9,2,1,63
5,1,University Departments of Anna University Chen...,EE,ELECTRICAL AND\nELECTRONICS ENGINEERING,19,18,2,13,9,2,1,64
6,1,University Departments of Anna University Chen...,EL,Electronics Engineering (VLSI\nDesign and Tech...,19,18,2,13,9,2,1,64
7,1,University Departments of Anna University Chen...,EM,ELECTRONICS AND\nCOMMUNICATION\nENGINEERING (SS),20,17,1,13,10,2,0,63
8,1,University Departments of Anna University Chen...,GI,GEO INFORMATICS,20,16,3,13,9,2,1,64
9,1,University Departments of Anna University Chen...,IE,INDUSTRIAL ENGINEERING,19,17,3,12,10,2,1,64


In [ ]:
print(len(set(df1['COLLEGE CODE']) - set(unique_clg_ppr_order)))
set(df1['COLLEGE CODE']) - set(unique_clg_ppr_order)

32


{'1102',
 '1124',
 '1150',
 '1209',
 '1232',
 '1235',
 '1321',
 '1335',
 '1415',
 '1438',
 '1511',
 '1529',
 '2',
 '2332',
 '2343',
 '2350',
 '2368',
 '2378',
 '2616',
 '3',
 '3766',
 '3808',
 '3848',
 '3923',
 '4024',
 '4925',
 '4932',
 '4998',
 '5012',
 '5537',
 '5851',
 '5924'}

In [ ]:
print(len(set(unique_clg_ppr_order) - set(df1['COLLEGE CODE'])))
set(unique_clg_ppr_order) - set(df1['COLLEGE CODE'])

20


{'1125',
 '1213',
 '1231',
 '1313',
 '1324',
 '1398',
 '1533',
 '2314',
 '2328',
 '2606',
 '2657',
 '2661',
 '2665',
 '2722',
 '2734',
 '3462',
 '3814',
 '4972',
 '4984',
 '4995'}

In [18]:
unique_clg_seat_matrix_2025 = pdf_dataframes[0]['COLLEGE CODE'].unique()

In [ ]:
type(cutoff_OC_CS_df_org['College Code'][0])

numpy.int64

In [19]:
seat_matrix_2025_sorted = pd.DataFrame()

for college_code in unique_clg_ppr_order:
    if college_code in unique_clg_seat_matrix_2025:
        curr_clg_courses = pdf_dataframes[0][pdf_dataframes[0]['COLLEGE CODE'] == college_code]['BRANCH CODE']
        total_seat_per_college = 0

        for curr_clg_course in curr_clg_courses:
            if college_code == '1':
                print(curr_clg_course in cs_branch_codes)
            if curr_clg_course in cs_branch_codes:
                seat_rows = pdf_dataframes[0][(pdf_dataframes[0]['COLLEGE CODE'] == college_code) & (pdf_dataframes[0]['BRANCH CODE'] == curr_clg_course)]
                total_seat_per_college += pd.to_numeric(seat_rows['OC'], errors='coerce').fillna(0).astype(int).sum() + pd.to_numeric(seat_rows['SC'], errors='coerce').fillna(0).astype(int).sum()
                if college_code == '1':
                    print(total_seat_per_college)

        # Get the College Name and Public Perception Rank safely
        college_name = pdf_dataframes[0].loc[pdf_dataframes[0]['COLLEGE CODE'] == college_code, 'COLLEGE NAME'].values[0]

        # Handle missing Public Perception Rank
        public_perception_rank = None
        rank_rows = cutoff_OC_CS_df_org[cutoff_OC_CS_df_org['College Code'] == int(college_code)]
        if not rank_rows.empty:
            public_perception_rank = rank_rows['PUBLIC PERCEPTION RANK'].values[0]

        seat_matrix_2025_sorted = pd.concat([
            seat_matrix_2025_sorted,
            pd.DataFrame([{
                'College Code': college_code,
                'College Name': college_name,
                'OC Average (2021-24)': rank_rows['OC AVG'].values[0] if not rank_rows.empty else 'N/A',
                'Total CS(related) seats': total_seat_per_college,
                'Public Perspective Rank': public_perception_rank if public_perception_rank is not None else 'N/A'
            }])
        ], ignore_index=True)


In [21]:
seat_matrix_2025_sorted = pd.DataFrame()

for college_code in unique_clg_ppr_order:
    if college_code in unique_clg_seat_matrix_2025:
        curr_clg_courses = pdf_dataframes[0][pdf_dataframes[0]['COLLEGE CODE'] == college_code]['BRANCH CODE']
        total_seat_per_college = {}
        for category in categories:
            total_seat_per_college[category] = 0

        for curr_clg_course in curr_clg_courses:
            if college_code == '1':
                print(curr_clg_course in cs_branch_codes)
            if curr_clg_course in cs_branch_codes:
                seat_rows = pdf_dataframes[0][(pdf_dataframes[0]['COLLEGE CODE'] == college_code) & (pdf_dataframes[0]['BRANCH CODE'] == curr_clg_course)]
                total_seat_per_college['OC'] += pd.to_numeric(seat_rows['OC'], errors='coerce').fillna(0).astype(int).sum()
                total_seat_per_college['BC'] += pd.to_numeric(seat_rows['BC'], errors='coerce').fillna(0).astype(int).sum()
                total_seat_per_college['BCM'] += pd.to_numeric(seat_rows['BCM'], errors='coerce').fillna(0).astype(int).sum()
                total_seat_per_college['MBC'] += pd.to_numeric(seat_rows['MBC'], errors='coerce').fillna(0).astype(int).sum()
                total_seat_per_college['SC'] += pd.to_numeric(seat_rows['SC'], errors='coerce').fillna(0).astype(int).sum()
                total_seat_per_college['SCA'] += pd.to_numeric(seat_rows['SCA'], errors='coerce').fillna(0).astype(int).sum()
                total_seat_per_college['ST'] += pd.to_numeric(seat_rows['ST'], errors='coerce').fillna(0).astype(int).sum()

                if college_code == '1':
                    print(total_seat_per_college)

        # Get the College Name and Public Perception Rank safely
        college_name = pdf_dataframes[0].loc[pdf_dataframes[0]['COLLEGE CODE'] == college_code, 'COLLEGE NAME'].values[0]

        # Handle missing Public Perception Rank
        public_perception_rank = None
        rank_rows = cutoff_OC_CS_df_org[cutoff_OC_CS_df_org['College Code'] == int(college_code)]
        if not rank_rows.empty:
            public_perception_rank = rank_rows['PUBLIC PERCEPTION RANK'].values[0]

        seat_matrix_2025_sorted = pd.concat([
            seat_matrix_2025_sorted,
            pd.DataFrame([{
                'College Code': college_code,
                'College Name': college_name,
                'Total OC seats': total_seat_per_college['OC'],
                'Total BC seats': total_seat_per_college['BC'],
                'Total BCM seats': total_seat_per_college['BCM'],
                'Total MBC seats': total_seat_per_college['MBC'],
                'Total SC seats': total_seat_per_college['SC'],
                'Total SCA seats': total_seat_per_college['SCA'],
                'Total ST seats': total_seat_per_college['ST'],
                'Public Perspective Rank': public_perception_rank if public_perception_rank is not None else 'N/A'
            }])
        ], ignore_index=True)


In [22]:
seat_matrix_2025_sorted['OC Cumulative Seats'] = seat_matrix_2025_sorted['Total OC seats'].cumsum()
seat_matrix_2025_sorted['BC Cumulative Seats'] = seat_matrix_2025_sorted['Total BC seats'].cumsum()
seat_matrix_2025_sorted['BCM Cumulative Seats'] = seat_matrix_2025_sorted['Total BCM seats'].cumsum()
seat_matrix_2025_sorted['MBC Cumulative Seats'] = seat_matrix_2025_sorted['Total MBC seats'].cumsum()
seat_matrix_2025_sorted['SC Cumulative Seats'] = seat_matrix_2025_sorted['Total SC seats'].cumsum()
seat_matrix_2025_sorted['SCA Cumulative Seats'] = seat_matrix_2025_sorted['Total SCA seats'].cumsum()
seat_matrix_2025_sorted['ST Cumulative Seats'] = seat_matrix_2025_sorted['Total ST seats'].cumsum()
seat_matrix_2025_sorted

,College Code,College Name,Total OC seats,Total BC seats,Total BCM seats,Total MBC seats,Total SC seats,Total SCA seats,Total ST seats,Public Perspective Rank,OC Cumulative Seats,BC Cumulative Seats,BCM Cumulative Seats,MBC Cumulative Seats,SC Cumulative Seats,SCA Cumulative Seats,ST Cumulative Seats
0,1,University Departments of Anna University Chen...,83,72,9,53,40,8,2,1,83,72,9,53,40,8,2
1,2006,PSG College of Technology (Autonomous) Peelame...,68,57,6,43,32,6,1,3,151,129,15,96,72,14,3
2,4,University Departments of Anna University Chen...,82,71,9,55,40,7,3,4,233,200,24,151,112,21,6
3,1315,Sri Sivasubramaniya Nadar College of Engineeri...,66,56,8,41,31,7,2,5,299,256,32,192,143,28,8
4,5008,Thiagarajar College of Engineering (Autonomous...,102,86,11,62,50,9,4,7,401,342,43,254,193,37,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,1445,"Sri Rangapoopathi College of Engineering,\nAla...",57,50,7,39,29,6,1,405,23447,20048,2643,15118,11352,2268,746
389,3854,Mahath Amma Institute of Engineering and\nTech...,41,37,5,28,20,4,1,406,23488,20085,2648,15146,11372,2272,747
390,1524,"Annamalaiar College Of Engineering, Chetpet-Po...",19,17,2,12,9,2,0,407,23507,20102,2650,15158,11381,2274,747
391,2341,"Ganesh College of Engineering, Attur Main Road...",49,44,6,33,23,5,2,408,23556,20146,2656,15191,11404,2279,749


In [23]:
seat_matrix_2025_sorted.head(10)

,College Code,College Name,Total OC seats,Total BC seats,Total BCM seats,Total MBC seats,Total SC seats,Total SCA seats,Total ST seats,Public Perspective Rank,OC Cumulative Seats,BC Cumulative Seats,BCM Cumulative Seats,MBC Cumulative Seats,SC Cumulative Seats,SCA Cumulative Seats,ST Cumulative Seats
0,1,University Departments of Anna University Chen...,83,72,9,53,40,8,2,1,83,72,9,53,40,8,2
1,2006,PSG College of Technology (Autonomous) Peelame...,68,57,6,43,32,6,1,3,151,129,15,96,72,14,3
2,4,University Departments of Anna University Chen...,82,71,9,55,40,7,3,4,233,200,24,151,112,21,6
3,1315,Sri Sivasubramaniya Nadar College of Engineeri...,66,56,8,41,31,7,2,5,299,256,32,192,143,28,8
4,5008,Thiagarajar College of Engineering (Autonomous...,102,86,11,62,50,9,4,7,401,342,43,254,193,37,12
5,2007,Coimbatore Institute of Technology (Autonomous...,53,45,6,33,25,4,3,8,454,387,49,287,218,41,15
6,2377,PSG Institute of Technology and Applied Resear...,54,47,6,36,25,5,3,9,508,434,55,323,243,46,18
7,2005,Government College of Technology (Autonomous)\...,34,29,4,22,16,3,1,10,542,463,59,345,259,49,19
8,1399,Chennai Institute of Technology (Autonomous)\n...,234,201,26,153,115,24,8,13,776,664,85,498,374,73,27
9,2712,"Kumaraguru College of Technology (Autonomous),...",102,85,11,63,49,10,3,14,878,749,96,561,423,83,30


In [24]:
seat_matrix_2025_sorted.to_excel("/content/drive/MyDrive/TNEA Seat Matrix 2025/seat_matrix_2024_all_Category_cumulative_seats.xlsx", index=False)


In [ ]:
seat_matrix_2025_CS_cumsum

,College Code,COLLEGE NAME,BRANCH CODE,BRANCH NAME,OC,BC,BCM,MBC,SC,SCA,ST,TOTAL
0,1399,Chennai Institute of Technology (Autonomous)\n...,AM,COMPUTER SCIENCE AND\nENGINEERING (AI AND\nMAC...,17.0,14.0,1.0,12.0,9.0,1.0,0.0,54.0
1,1432,Rajalakshmi Institute of Technology (Autonomou...,AM,COMPUTER SCIENCE AND\nENGINEERING (AI AND\nMAC...,33.0,29.0,3.0,23.0,17.0,2.0,1.0,108.0
2,2764,KPR Institute of Engineering and Technology\n(...,AM,COMPUTER SCIENCE AND\nENGINEERING (AI AND\nMAC...,54.0,47.0,6.0,37.0,28.0,4.0,2.0,178.0
3,2722,NaN,NaN,NaN,54.0,47.0,6.0,37.0,28.0,4.0,2.0,178.0
4,2739,Sri Eshwar College of Engineering (Autonomous)...,AM,COMPUTER SCIENCE AND\nENGINEERING (AI AND\nMAC...,75.0,65.0,9.0,51.0,39.0,6.0,3.0,248.0
...,...,...,...,...,...,...,...,...,...,...,...,...
88,3860,St.Anne's College of Engineering and Technolog...,AM,COMPUTER SCIENCE AND\nENGINEERING (AI AND\nMAC...,1069.0,908.0,125.0,687.0,520.0,106.0,36.0,3451.0
89,1518,Thiruvalluvar College of Engineering and Techn...,AM,COMPUTER SCIENCE AND\nENGINEERING (AI AND\nMAC...,1078.0,916.0,125.0,692.0,524.0,107.0,36.0,3478.0
90,2705,"CSI College of Engineering, Ketti Valley, The ...",AM,COMPUTER SCIENCE AND\nENGINEERING (AI AND\nMAC...,1092.0,928.0,127.0,701.0,530.0,108.0,37.0,3523.0
91,2646,Tagore Institute of Engineering and Technology...,AM,COMPUTER SCIENCE AND\nENGINEERING (AI AND\nMAC...,1100.0,934.0,128.0,706.0,535.0,109.0,37.0,3549.0


In [ ]:
gen_rank_list_2025 = pd.DataFrame()

# Extract tables and merge into one DataFrame
with pdfplumber.open('/content/drive/MyDrive/TNEA Seat Matrix 2025/ACADEMIC_GENERAL_RANK_LIST_2025.pdf') as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            df = pd.DataFrame(table[1:], columns=table[0])  # Skip header row
            gen_rank_list_2025 = pd.concat([gen_rank_list_2025, df], ignore_index=True)

# Append the combined DataFrame to the list

print(f'Loaded: ACADEMIC_GENERAL_RANK_LIST_2025.pdf → {gen_rank_list_2025.shape[0]} rows')

In [ ]:
gen_rank_list_2025 = pd.read_excel('/content/drive/MyDrive/TNEA Seat Matrix 2025/ACADEMIC_GENERAL_RANK_LIST_2025.pdf')
gen_rank_list_2025

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
seat_matrix_2025_sorted.to_excel("/content/drive/MyDrive/TNEA Seat Matrix 2025/seat_matrix_2025_cumulative_seats.xlsx", index=False)